In [1]:
import os
from openai import OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [2]:
directorio = "C:\\Users\\crebr\\Rag Manuals\\"


todos_los_documentos = []

for archivo in os.listdir(directorio):
    if archivo.endswith(".pdf"):
        ruta_pdf = os.path.join(directorio, archivo)
        print(f"Cargando: {ruta_pdf}")
        loader = PyPDFLoader(ruta_pdf)
        documentos = loader.load()
        todos_los_documentos.extend(documentos)

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
fragmentos = splitter.split_documents(todos_los_documentos)

embeddings = OpenAIEmbeddings(api_key="APIKEY") 
vectorstore = Chroma.from_documents(fragmentos, embeddings, persist_directory="./db")

vectorstore.persist()

Cargando: C:\Users\crebr\Rag Manuals\eng.pdf
Cargando: C:\Users\crebr\Rag Manuals\MFL67105007_english.pdf


C:\Users\crebr\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
# Recuperamos la base vectorial
vectorstore = Chroma(persist_directory="./db", embedding_function=embeddings)

In [9]:
# Pregunta del usuario
pregunta = "¿What kind of cables I have  in Model BE 320 SD?"

# Recuperar los 3 fragmentos más parecidos
docs_relevantes = vectorstore.similarity_search(pregunta, k=3)
contexto = "\n\n".join([doc.page_content for doc in docs_relevantes])

In [10]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="APIKEY",
)
prompt = f"""
Usa el siguiente contexto para responder la pregunta. Si no puedes responder con lo que ves aquí, di que no tienes suficiente información.

Contexto:
{contexto}

Pregunta:
{pregunta}
"""

# Llamar a ChatGPT
respuesta = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.2
)

print(respuesta.choices[0].message.content.strip())

The cables included in the Model BE 320 SD are a power cable and an RGB cable. An RS232C cable can also be used, but it is optional and not included.
